In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **CS224W - Colab 1**

In this Colab, we will write a full pipeline for **learning node embeddings**.
We will go through the following 3 steps.  
在这个 Colab 中，我们将编写一个完整的管道来学习节点嵌入。 我们将经历以下 3 个步骤。

To start, we will load a classic graph in network science, the [Karate Club Network](https://en.wikipedia.org/wiki/Zachary%27s_karate_club). We will explore multiple graph statistics for that graph.  
首先，我们将加载网络科学中的经典图，即空手道俱乐部网络。 我们将探索该图的多个图统计信息。

We will then work together to transform the graph structure into a PyTorch tensor, so that we can perform machine learning over the graph.  
然后我们将共同将图结构转换为 PyTorch 张量，以便我们可以对图执行机器学习。

Finally, we will finish the first learning algorithm on graphs: a node embedding model. For simplicity, our model here is simpler than DeepWalk / node2vec algorithms taught in the lecture. But it's still rewarding and challenging, as we will write it from scratch via PyTorch.  
最后，我们将完成第一个图学习算法：节点嵌入模型。 为简单起见，我们这里的模型比讲座中教授的 DeepWalk / node2vec 算法简单。 但它仍然是有益的和具有挑战性的，因为我们将通过 PyTorch 从头开始编写它。

Now let's get started!  
现在让我们开始吧！

**Note**: Make sure to **sequentially run all the cells**, so that the intermediate variables / packages will carry over to the next cell  
注意：确保顺序运行所有单元格，以便中间变量/包会延续到下一个单元格

# 1 Graph Basics
To start, we will load a classic graph in network science, the [Karate Club Network](https://en.wikipedia.org/wiki/Zachary%27s_karate_club). We will explore multiple graph statistics for that graph.  
首先，我们将加载网络科学中的经典图，即空手道俱乐部网络。 我们将探索该图的多个图统计信息。

## Setup
We will heavily use NetworkX in this Colab.

In [ ]:
import networkx as nx

## Zachary's karate club network

The [Karate Club Network](https://en.wikipedia.org/wiki/Zachary%27s_karate_club) is a graph describes a social network of 34 members of a karate club and documents links between members who interacted outside the club.  
空手道俱乐部网络是一个图表，描述了一个空手道俱乐部的 34 名成员的社交网络，并记录了在俱乐部外互动的成员之间的链接。

In [ ]:
G = nx.karate_club_graph()

# G is an undirected graph
type(G)

In [ ]:
# Visualize the graph
nx.draw(G, with_labels = True)

## Question 1: What is the average degree of the karate club network? (5 Points)  
问题1：空手道俱乐部网络的平均度数是多少？ (5 分)

In [ ]:
def average_degree(num_edges, num_nodes):
  # TODO: Implement this function that takes number of edges
  # and number of nodes, and returns the average node degree of 
  # the graph. Round the result to nearest integer (for example 
  # 3.3 will be rounded to 3 and 3.7 will be rounded to 4)
  # 实现此函数，该函数采用边数和节点数，并返回图的平均节点度。 将结果四舍五入到最接近的整数（例如 3.3 将四舍五入为 3，3.7 将四舍五入为 4）

  avg_degree = 0

  ############# Your code here ############
  '''
  if (num_edges / num_nodes) - int(num_edges / num_nodes) > 0.5:
      avg_degree = int(num_edges / num_nodes) + 1
  else:
      avg_degree = int(num_edges / num_nodes)
  '''
  avg_degree = 2*num_edges / num_nodes
  avg_degree = round(avg_degree, 0)
  #########################################

  return avg_degree

num_edges = G.number_of_edges()
num_nodes = G.number_of_nodes()
avg_degree = average_degree(num_edges, num_nodes)
print("Average degree of karate club network is {}".format(avg_degree))

## Question 2: What is the average clustering coefficient of the karate club network? (5 Points)  
问题2：空手道俱乐部网络的平均聚类系数是多少？ (5 分)

In [ ]:
def average_clustering_coefficient(G):
  # TODO: Implement this function that takes a nx.Graph
  # and returns the average clustering coefficient. Round 
  # the result to 2 decimal places (for example 3.333 will
  # be rounded to 3.33 and 3.7571 will be rounded to 3.76)

  avg_cluster_coef = 0

  ############# Your code here ############
  ## Note: 
  ## 1: Please use the appropriate NetworkX clustering function
  num_nodes = G.number_of_nodes()
  avg_cluster_coef = nx.average_clustering(G, nodes=None, weight=None, count_zeros=True)
  # print(avg_cluster_coef)
  '''
  if avg_cluster_coef*1000%5 > 1:
      avg_cluster_coef = int((avg_cluster_coef*100 + 1))/100
  else:
      avg_cluster_coef = int(avg_cluster_coef*100)/100
  '''
  avg_cluster_coef = round(avg_cluster_coef, 2)
  #########################################

  return avg_cluster_coef

avg_cluster_coef = average_clustering_coefficient(G)
print("Average clustering coefficient of karate club network is {}".format(avg_cluster_coef))

## Question 3: What is the PageRank value for node 0 (node with id 0) after one PageRank iteration? (5 Points)
问题 3：经过一次 PageRank 迭代后，节点 0（id 为 0 的节点）的 PageRank 值是多少？ (5 分)

Please complete the code block by implementing the PageRank equation:  
请通过实现 PageRank 方程来完成代码块：  
 $r_j = \sum_{i \rightarrow j} \beta \frac{r_i}{d_i} + (1 - \beta) \frac{1}{N}$

In [ ]:
def one_iter_pagerank(G, beta, r0, node_id):
  # TODO: Implement this function that takes a nx.Graph, beta, r0 and node id.
  # The return value r1 is one interation PageRank value for the input node.
  # Please round r1 to 2 decimal places.

  r1 = 0

  ############# Your code here ############
  ## Note: 
  ## 1: You should not use nx.pagerank
  #得到node0的邻居→得到这些邻居的出度（是无向图，所以就是度数）→计算得到右式中的第一项（\sum{i→j}\beta\frac{r_i}{d_i}）→计算得到r_j
  for ni in nx.neighbors(G,node_id):
      #得到的每一个ni都是node0邻居的索引，参考：https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.neighbors.html
      #另一种获取节点邻居的方式是访问邻接矩阵，参考该博文notes部分：https://www.neusncp.com/user/blog?id=38
      di=G.degree[ni]  #获取node_ni的度数
      r1 += beta*float(r0/di)
  r1 +=  (1-beta)*r0
  print(r1)  
  r1 = round(r1, 2)
  '''
  pagerank_list = nx.pagerank(G, alpha=1)
  # print(pagerank_list)
  r1 = round(pagerank_list[node_id],2)
  '''
  #########################################

  return r1

beta = 0.8
r0 = 1 / G.number_of_nodes()
node = 0
r1 = one_iter_pagerank(G, beta, r0, node)
print("The PageRank value for node 0 after one iteration is {}".format(r1))

## Question 4: What is the (raw) closeness centrality for the karate club network node 5? (5 Points)  
空手道俱乐部网络节点 5 的（原始）接近中心性是什么？ (5 分)

The equation for closeness centrality is 接近中心性的方程是 $c(v) = \frac{1}{\sum_{u \neq v}\text{shortest path length between } u \text{ and } v}$

In [ ]:
def closeness_centrality(G, node=5):
  # TODO: Implement the function that calculates closeness centrality 
  # for a node in karate club network. G is the input karate club 
  # network and node is the node id in the graph. Please round the 
  # closeness centrality result to 2 decimal places.

  closeness = 0

  ## Note:
  ## 1: You can use networkx closeness centrality function.
  ## 2: Notice that networkx closeness centrality returns the normalized 
  ## closeness directly, which is different from the raw (unnormalized) 
  ## one that we learned in the lecture.
  '''
    node_length_pairs=nx.shortest_path_length(G,source=5)
    #返回字典，key是节点索引，value是source与该节点间的最短路径长度

    denominator=0  #分母
    for i in range(G.number_of_nodes()):
    if i!=5:  #其实不用这个判断也行，如果i=5就会是0
        denominator+=node_length_pairs[i]
    closeness=round(1/denominator,2)
  '''
  
  closeness = nx.closeness_centrality(G,u=node) # 函数是原始closeness centrality做了规范化（乘以 (图节点数量-1) ）
  closeness=closeness/(G.number_of_nodes()-1)
  closeness = round(closeness, 2)
  #########################################[node]

  return closeness

node = 5
closeness = closeness_centrality(G, node=node)
print("The karate club network has closeness centrality {}".format(closeness))

# 2 Graph to Tensor
We will then work together to transform the graph $G$ into a PyTorch tensor, so that we can perform machine learning over the graph.
然后我们将共同将图 G 转换为 PyTorch 张量，以便我们可以对图执行机器学习。

## Setup
Check if PyTorch is properly installed

In [ ]:
import torch
print(torch.__version__)

## PyTorch tensor basics

We can generate PyTorch tensor with all zeros, ones or random values.  
我们可以生成全为零、一或随机值的 PyTorch 张量。

In [ ]:
# Generate 3 x 4 tensor with all ones
ones = torch.ones(3, 4)
print(ones)

# Generate 3 x 4 tensor with all zeros
zeros = torch.zeros(3, 4)
print(zeros)

# Generate 3 x 4 tensor with random values on the interval [0, 1)
random_tensor = torch.rand(3, 4)
print(random_tensor)

# Get the shape of the tensor
print(ones.shape)

PyTorch tensor contains elements for a single data type, the `dtype`.  
PyTorch 张量包含单一数据类型 dtype 的元素。

In [ ]:
# Create a 3 x 4 tensor with all 32-bit floating point zeros
zeros = torch.zeros(3, 4, dtype=torch.float32)
print(zeros.dtype)

# Change the tensor dtype to 64-bit integer
zeros = zeros.type(torch.long)
print(zeros.dtype)

## Question 5: Getting the edge list of the karate club network and transform it into `torch.LongTensor`. What is the `torch.sum` value of `pos_edge_index` tensor? (10 Points)  
问题5：获取空手道俱乐部网络的边缘列表，并转化为torch.LongTensor。 pos_edge_index 张量的 torch.sum 值是多少？ （10分）

In [ ]:
def graph_to_edge_list(G):
  # TODO: Implement the function that returns the edge list of
  # an nx.Graph. The returned edge_list should be a list of tuples
  # where each tuple is a tuple representing an edge connected 
  # by two nodes.

  edge_list = []

  ############# Your code here ############
  for edge in G.edges():
      edge_list.append(edge)
  #########################################

  return edge_list

def edge_list_to_tensor(edge_list):
  # TODO: Implement the function that transforms the edge_list to
  # tensor. The input edge_list is a list of tuples and the resulting
  # tensor should have the shape [2 x len(edge_list)].

  edge_index = torch.tensor([])

  ############# Your code here ############
  edge_index = torch.LongTensor(edge_list).t()
  # edge_index = torch.tensor(edge_list).transpose(1, 0)
  #########################################

  return edge_index

pos_edge_list = graph_to_edge_list(G)
pos_edge_index = edge_list_to_tensor(pos_edge_list)
print("The pos_edge_index tensor has shape {}".format(pos_edge_index.shape))
print("The pos_edge_index tensor has sum value {}".format(torch.sum(pos_edge_index)))

## Question 6: Please implement following function that samples negative edges. Then you will answer which edges (edge_1 to edge_5) can be negative ones in the karate club network? (10 Points)  
问题 6：请实现以下对负边缘进行采样的函数。 然后你会回答空手道俱乐部网络中哪些边（edge_1 到 edge_5）可以是负的？ （10分）

In [ ]:
import random

def sample_negative_edges(G, num_neg_samples):
  # TODO: Implement the function that returns a list of negative edges.
  # The number of sampled negative edges is num_neg_samples. You do not
  # need to consider the corner case when the number of possible negative edges
  # is less than num_neg_samples. It should be ok as long as your implementation 
  # works on the karate club network. In this implementation, self loop should 
  # not be considered as either a positive or negative edge. Also, notice that 
  # the karate club network is an undirected graph, if (0, 1) is a positive 
  # edge, do you think (1, 0) can be a negative one?

  neg_edge_list = []

  ############# Your code here ############
  #得到图中所有不存在的边（这个函数只会返回一侧，不会出现逆边）
  non_edges_one_side=list(enumerate(nx.non_edges(G)))
  neg_edge_list_indices=random.sample(range(0,len(non_edges_one_side)),num_neg_samples)  #取样num_neg_samples长度的索引
  #抽取逻辑是按照non_edges_one_side的索引来抽取边
  for i in neg_edge_list_indices:
    neg_edge_list.append(non_edges_one_side[i][1])
  #########################################

  return neg_edge_list

# Sample 78 negative edges
neg_edge_list = sample_negative_edges(G, len(pos_edge_list))

# Transform the negative edge list to tensor
neg_edge_index = edge_list_to_tensor(neg_edge_list)
print("The neg_edge_index tensor has shape {}".format(neg_edge_index.shape))

# Which of following edges can be negative ones?
edge_1 = (7, 1)
edge_2 = (1, 33)
edge_3 = (33, 22)
edge_4 = (0, 4)
edge_5 = (4, 2)

############# Your code here ############
## Note:
## 1: For each of the 5 edges, print whether it can be negative edge
#如果边在图中，就认为不行
print('edge_1'+(" can't" if G.has_edge(edge_1[0],edge_1[1]) else ' can')+' be negative edge')
print('edge_2'+(" can't" if G.has_edge(edge_2[0],edge_2[1]) else ' can')+' be negative edge')
print('edge_3'+(" can't" if G.has_edge(edge_3[0],edge_3[1]) else ' can')+' be negative edge')
print('edge_4'+(" can't" if G.has_edge(edge_4[0],edge_4[1]) else ' can')+' be negative edge')
print('edge_5'+(" can't" if G.has_edge(edge_5[0],edge_5[1]) else ' can')+' be negative edge')
#########################################

# 3 Node Emebedding Learning

Finally, we will finish the first learning algorithm on graphs: a node embedding model.  
最后，我们将完成第一个图学习算法：节点嵌入模型。


## Setup

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

print(torch.__version__)

To write our own node embedding learning methods, we'll heavily use the [`nn.Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) module in PyTorch. Let's see how to use `nn.Embedding`:  
为了编写我们自己的节点嵌入学习方法，我们将大量使用 PyTorch 中的 nn.Embedding 模块。 让我们看看如何使用 nn.Embedding：

In [ ]:
# Initialize an embedding layer
# Suppose we want to have embedding for 4 items (e.g., nodes)
# Each item is represented with 8 dimensional vector

emb_sample = nn.Embedding(num_embeddings=4, embedding_dim=8)
print('Sample embedding layer: {}'.format(emb_sample))

We can select items from the embedding matrix, by using Tensor   
我们可以通过使用张量索引从嵌入矩阵中选择项目

In [ ]:
# Select an embedding in emb_sample
id = torch.LongTensor([1])
print(emb_sample(id))

# Select multiple embeddings
ids = torch.LongTensor([1, 3])
print(emb_sample(ids))

# Get the shape of the embedding weight matrix
shape = emb_sample.weight.data.shape
print(shape)

# Overwrite the weight to tensor with all ones
emb_sample.weight.data = torch.ones(shape)
print(emb_sample.weight.data)

# Let's check if the emb is indeed initilized
ids = torch.LongTensor([0, 3])
print(emb_sample(ids))

Now, it's your time to create node embedding matrix for the graph we have!  
现在，是时候为我们拥有的图形创建节点嵌入矩阵了！
- We want to have **16 dimensional** vector for each node in the karate club network.  
我们希望空手道俱乐部网络中的每个节点都有 16 维向量。
- We want to initalize the matrix under **uniform distribution**, in the range of $[0, 1)$. We suggest you using [`torch.rand`](https://pytorch.org/docs/stable/generated/torch.rand.html).  
我们想在均匀分布下初始化矩阵，在 [0,1) 的范围内。 我们建议您使用 torch.rand。

In [ ]:
# Please do not change / reset the random seed
torch.manual_seed(1)

def create_node_emb(num_node=34, embedding_dim=16):
  # TODO: Implement this function that will create the node embedding matrix.
  # A torch.nn.Embedding layer will be returned. You do not need to change 
  # the values of num_node and embedding_dim. The weight matrix of returned 
  # layer should be initialized under uniform distribution. 

  emb = None

  ############# Your code here ############
  emb = nn.Embedding(num_embeddings=num_node, embedding_dim=embedding_dim)
  #Embedding初始化本来就是均匀分布。不过在这里应该是要用manual_seed来维持可复现性
  emb.weight.data=torch.rand(num_node,embedding_dim)
  #########################################

  return emb

emb = create_node_emb()
ids = torch.LongTensor([0, 3])

# Print the embedding layer
print("Embedding: {}".format(emb))

# An example that gets the embeddings for node 0 and 3
print(emb(ids))

## Visualize the initial node embeddings
One good way to understand an embedding matrix, is to visualize it in a 2D space.
Here, we have implemented an embedding visualization function for you.
We first do PCA to reduce the dimensionality of embeddings to a 2D space.
Then visualize each point, colored by the community it belongs to.  
理解嵌入矩阵的一种好方法是在 2D 空间中对其进行可视化。 在这里，我们为您实现了嵌入可视化功能。 我们首先进行 PCA 以将嵌入的维度降低到 2D 空间。 然后可视化每个点，由它所属的社区着色。

In [ ]:
def visualize_emb(emb):
  X = emb.weight.data.numpy()
  pca = PCA(n_components=2)
  components = pca.fit_transform(X)
  plt.figure(figsize=(6, 6))
  club1_x = []
  club1_y = []
  club2_x = []
  club2_y = []
  for node in G.nodes(data=True):
    if node[1]['club'] == 'Mr. Hi':
      club1_x.append(components[node[0]][0])
      club1_y.append(components[node[0]][1])
    else:
      club2_x.append(components[node[0]][0])
      club2_y.append(components[node[0]][1])
  plt.scatter(club1_x, club1_y, color="red", label="Mr. Hi")
  plt.scatter(club2_x, club2_y, color="blue", label="Officer")
  plt.legend()
  plt.show()

# Visualize the initial random embeddding
visualize_emb(emb)

## Question 7: Training the embedding! What is the best performance you can get? Please report both the best loss and accuracy on Gradescope. (20 Points)  
问题 7：训练嵌入！ 您可以获得的最佳性能是什么？ 请在 Gradescope 上报告最佳损失和准确度。 (20 分)

In [ ]:
from torch.optim import SGD

def accuracy(pred, label):
  # TODO: Implement the accuracy function. This function takes the 
  # pred tensor (the resulting tensor after sigmoid) and the label 
  # tensor (torch.LongTensor). Predicted value greater than 0.5 will 
  # be classified as label 1. Else it will be classified as label 0.
  # The returned accuracy should be rounded to 4 decimal places. 
  # For example, accuracy 0.82956 will be rounded to 0.8296.

  accu = 0.0

  ############# Your code here ############
  #accuracy=预测与实际一致的结果数/所有结果数
  #pred tensor和label tensor都是[78*2(156)]大小的tensor
  pred = pred > 0.5
  accu = (pred==label).sum().item() / (pred.shape[0])
  accu = round(accu,4)
  #########################################

  return accu

def train(emb, loss_fn, sigmoid, train_label, train_edge):
  # TODO: Train the embedding layer here. You can also change epochs and 
  # learning rate. In general, you need to implement: 
  # (1) Get the embeddings of the nodes in train_edge
  # (2) Dot product the embeddings between each node pair
  # (3) Feed the dot product result into sigmoid
  # (4) Feed the sigmoid output into the loss_fn
  # (5) Print both loss and accuracy of each epoch 
  # (as a sanity check, the loss should decrease during training)

  epochs = 500
  learning_rate = 0.1

  optimizer = SGD(emb.parameters(), lr=learning_rate, momentum=0.9)

  for i in range(epochs):
    ############# Your code here ############
    optimizer.zero_grad()
    train_node_emb = emb(train_edge)  #[2,156,16]
    dot_product_result = train_node_emb[0].mul(train_node_emb[1])  #点对之间对应位置嵌入相乘，[156,16]
    dot_product_result = torch.sum(dot_product_result,1)  #加起来，构成点对之间向量的点积，[156]
    sigmoid_result = sigmoid(dot_product_result)  #将这个点积结果经过激活函数映射到0,1之间
    loss_result = loss_fn(sigmoid_result,train_label)
    loss_result.backward()
    optimizer.step()
    if i%10 == 0:  #其实这个应该每一轮都打印一遍的，但是我嫌太大了就十轮打印一遍了
      print(loss_result)
      print(accuracy(sigmoid_result,train_label))
    #########################################

loss_fn = nn.BCELoss()
sigmoid = nn.Sigmoid()

# Generate the positive and negative labels
pos_label = torch.ones(pos_edge_index.shape[1], )
neg_label = torch.zeros(neg_edge_index.shape[1], )

# Concat positive and negative labels into one tensor
train_label = torch.cat([pos_label, neg_label], dim=0)

# Concat positive and negative edges into one tensor
# Since the network is very small, we do not split the edges into val/test sets
train_edge = torch.cat([pos_edge_index, neg_edge_index], dim=1)

train(emb, loss_fn, sigmoid, train_label, train_edge)

## Visualize the final node embeddings
Visualize your final embedding here! 
You can visually compare the figure with the previous embedding figure. 
After training, you should oberserve that the two classes are more evidently separated. 
This is a great sanitity check for your implementation as well.  
在此处可视化您的最终嵌入！ 您可以直观地将图与之前的嵌入图进行比较。 训练后，您应该观察到两个类的分离更加明显。 这对您的实施也是一个很好的健全性检查。

In [ ]:
# Visualize the final learned embedding
visualize_emb(emb)

# Submission

In order to get credit, you must go submit your answers on Gradescope.